In [1]:
import numpy as np
import pandas as pd
import json
import gc
from collections import defaultdict
import sys
from enum import Enum
import random
import pickle

random.seed(824)

print("Setup complete!")

Setup complete!


In [2]:
ml_ratings_file = "rating_features-small.csv"
ratings_df = pd.read_csv(ml_ratings_file)
ratings_df

,Unnamed: 0,userId,movieId,rating
0,4446,1,32,5.0
1,4447,1,253,5.0
2,4448,1,260,5.0
3,4449,1,292,3.0
4,4450,1,316,5.0
...,...,...,...,...
723899,24982749,4891,5992,4.0
723900,24982750,4891,6027,5.0
723901,24982751,4891,6242,3.5
723902,24982753,4891,6345,4.0


In [3]:
users = set(ratings_df["userId"])
num_of_users = len(users)
num_of_users

4891

In [4]:
user_rating_counts = ratings_df["userId"].value_counts().rename_axis('userId').reset_index(name='counts')
user_rating_counts

,userId,counts
0,4508,3507
1,880,3346
2,2550,2964
3,4462,2627
4,3346,2565
...,...,...
4886,4082,18
4887,1102,18
4888,3659,15
4889,3762,15


In [5]:
valid = True
def check_valid_users(row):
    if row["counts"] < 5:
        print(row["userId"])
        valid = False
user_rating_counts.apply(check_valid_users, axis=1)
print(valid)

True


In [6]:

class Split(Enum):
    TRAIN = 1
    VALID = 2
    TEST = 3

def build_edges(row):
    user_id = int(row["userId"])
    movie_id = int(row["movieId"])
    user = user_id - 1
    movie = movie_map[movie_id]
    rating = row["rating"]
    split = random.choices(list(Split), weights=(70,10,20), k=1)[0].value
    movie_adj_list_test[movie].add((user, rating))
    user_adj_list_test[user].add((movie, rating))
    if split == Split.TEST.value:
        edge_list_test.append((user, movie, rating))
    elif split == Split.VALID.value:
        movie_adj_list_valid[movie].add((user, rating))
        user_adj_list_valid[user].add((movie, rating))
        edge_list_valid.append((user, movie, rating))
    elif split == Split.TRAIN.value:
        movie_adj_list_valid[movie].add((user, rating))
        user_adj_list_valid[user].add((movie, rating))
        movie_adj_list_train[movie].add((user, rating))
        user_adj_list_train[user].add((movie, rating))
        edge_list_train.append((user, movie, rating))
    

In [7]:
ml_movies_file = "movie_features-small.csv"
movies_df = pd.read_csv(ml_movies_file)

user_feat = []
movie_map = {} # map movieId to position in movies_feat

movies_feat = movies_df.iloc[:, 1:].to_numpy().tolist()

i = 0
for idx in movies_df.movieId:
	movie_map[int(idx)] = i
	i += 1

del movies_df
gc.collect()

num_of_movies = len(movies_feat)

#ml_ratings_file = "rating_features1.csv"
#ratings_df = pd.read_csv(ml_ratings_file)

movie_adj_list_train = [set() for _ in range(num_of_movies)]
movie_adj_list_valid = [set() for _ in range(num_of_movies)]
movie_adj_list_test = [set() for _ in range(num_of_movies)]
user_adj_list_train = [set() for _ in range(num_of_users)]
user_adj_list_valid = [set() for _ in range(num_of_users)]
user_adj_list_test = [set() for _ in range(num_of_users)]
edge_list_train = []
edge_list_valid = []
edge_list_test = []

ratings_df.apply(build_edges, axis=1)

del ratings_df
gc.collect()



0

In [8]:
print(len(movies_feat))
print(len(movies_feat[0]))
print(len(movie_adj_list_train))
print(len(movie_adj_list_valid))
print(len(movie_adj_list_test))
print(len(user_adj_list_train))
print(len(user_adj_list_valid))
print(len(user_adj_list_test))
print(len(edge_list_train))
print(len(edge_list_valid))
print(len(edge_list_test))

8726
4564
8726
8726
8726
4891
4891
4891
506526
72332
145046


In [14]:
def find_in_valid(movie):
    j = 0
    user = None
    rating = None
    for edge in edge_list_valid:
        if movie == edge[1]:
            user = edge[0]
            rating = edge[2]
            break
        j += 1
    return (user, movie, rating)

def find_in_test(movie):
    j = 0
    user = None
    rating = None
    for edge in edge_list_test:
        if movie == edge[1]:
            user = edge[0]
            rating = edge[2]
            break
        j += 1
    return (user, movie, rating)

def update_valid(el):
    edge_list_valid.remove(el)
    edge_list_train.append(el)
    movie_adj_list_train[el[1]].add((el[0], el[2]))
    user_adj_list_train[el[0]].add((el[1], el[2]))

def update_test(el):
    edge_list_test.remove(el)
    edge_list_train.append(el)
    movie_adj_list_train[el[1]].add((el[0], el[2]))
    user_adj_list_train[el[0]].add((el[1], el[2]))

neg_idx = []
i = 0
for neigh in movie_adj_list_train:
    if len(neigh) < 2:
        neg_idx.append(i)
    i += 1
print(neg_idx)


[]


In [13]:
for idx in neg_idx:
    print("For idx")
    edge1 = find_in_test(idx)
    print(f"Edge 1: {edge1}")
    if edge1[0] != None:
        update_test(edge1)
    if len(movie_adj_list_train[idx]) < 2:
        edge2 = find_in_valid(idx)
        print(f"Edge 2: {edge2}")
        update_valid(edge2)

For idx
Edge 1: (None, 5268, None)
Edge 2: (2382, 5268, 3.0)
For idx
Edge 1: (373, 5322, 3.0)
For idx
Edge 1: (946, 5629, 1.0)
For idx
Edge 1: (4507, 5676, 3.5)
Edge 2: (1346, 5676, 2.0)
For idx
Edge 1: (2525, 5704, 5.0)
For idx
Edge 1: (2171, 5716, 3.0)
For idx
Edge 1: (3051, 5858, 4.0)
For idx
Edge 1: (879, 5987, 2.5)
For idx
Edge 1: (951, 6196, 0.5)
For idx
Edge 1: (879, 6550, 1.5)
For idx
Edge 1: (879, 6771, 2.5)
For idx
Edge 1: (879, 6839, 3.0)
For idx
Edge 1: (713, 7286, 3.5)
For idx
Edge 1: (135, 7298, 3.0)
For idx
Edge 1: (3044, 7455, 5.0)
For idx
Edge 1: (510, 7577, 3.0)
For idx
Edge 1: (946, 7636, 4.5)
For idx
Edge 1: (879, 7833, 2.5)
For idx
Edge 1: (879, 8119, 1.5)
For idx
Edge 1: (1175, 8186, 3.0)
For idx
Edge 1: (713, 8196, 2.5)
For idx
Edge 1: (370, 8505, 5.0)


In [15]:
def find_in_valid(user):
    j = 0
    user = None
    rating = None
    for edge in edge_list_valid:
        if user == edge[0]:
            movie = edge[1]
            rating = edge[2]
            break
        j += 1
    return (user, movie, rating)

def find_in_test(user):
    j = 0
    user = None
    rating = None
    for edge in edge_list_test:
        if user == edge[0]:
            movie = edge[1]
            rating = edge[2]
            break
        j += 1
    return (user, movie, rating)

neg_idx = []
i = 0
for neigh in user_adj_list_train:
    if len(neigh) < 2:
        neg_idx.append(i)
    i += 1
print(neg_idx)

[]


In [40]:
train = 0
valid = 0
test = 0
for i in range(100):
    split = random.choices(list(Split), weights=(70,10,20), k=1)[0].value
    if split == Split.TRAIN.value:
       train += 1
    elif split == Split.VALID.value:
        valid += 1
    elif split == Split.TEST.value:
        test += 1
print(f"train: {train}")
print(f"valid: {valid}")
print(f"test: {test}")

train: 71
valid: 8
test: 21


In [16]:
data = dict()
data["movies_feat"] = movies_feat
data["movie_map"] = movie_map
data["movie_adj_list_train"] = movie_adj_list_train
data["movie_adj_list_valid"] = movie_adj_list_valid
data["movie_adj_list_test"] = movie_adj_list_test
data["user_adj_list_train"] = user_adj_list_train
data["user_adj_list_valid"] = user_adj_list_valid
data["user_adj_list_test"] = user_adj_list_test
data["edge_list_train"] = edge_list_train
data["edge_list_valid"] = edge_list_valid
data["edge_list_test"] = edge_list_test

a_file = open("prepared_data/data-small.pkl", "wb")
pickle.dump(data, a_file)
a_file.close()


In [8]:
a_file = open("prepared_data/data1.pkl", "rb")
data = pickle.load(a_file)
a_file.close()
movies_feat = data["movies_feat"]
movie_map = data["movie_map"]
movie_adj_list = data["movie_adj_list"]
user_adj_list = data["user_adj_list"]
edge_list = data["edge_list"]

In [15]:
print(len(user_adj_list))

162541


In [6]:
list(Split)

[<Split.TRAIN: 1>, <Split.VALID: 2>, <Split.TEST: 3>]

In [4]:
dictionary_data = {"a": [4,5], "b": {"g": [1,7]}}

a_file = open("prepared_data/data.pkl", "wb")
pickle.dump(dictionary_data, a_file)
a_file.close()

a_file = open("prepared_data/data.pkl", "rb")
output = pickle.load(a_file)
print(output)
a_file.close()

{'a': [4, 5], 'b': {'g': [1, 7]}}
